In [1]:
from nltk.parse.stanford import StanfordDependencyParser

In [2]:
import nltk
from nltk.corpus import sentiwordnet as swn
import os
import sys

In [3]:
import os
from nltk.parse.stanford import StanfordParser

In [4]:
path_to_jar = 'C:/Users/Default.migrated/AppData/Local/stanford-parser/3.5.2/libexec/stanford-parser.jar'

In [5]:
path_to_models_jar = 'C:/Users/Default.migrated/AppData/Local/stanford-parser/3.5.2/libexec/stanford-parser-3.5.2-models.jar'

In [6]:
dependency_parser = StanfordDependencyParser(path_to_jar=path_to_jar, path_to_models_jar=path_to_models_jar)

In [7]:
def score(p):
    s = swn.senti_synsets(p)
    p = len(s)
    pos_score=0
    neg_score=0
    for i in s:
        pos_score = pos_score + i.pos_score()
        neg_score = neg_score + i.neg_score()
    if p == 0:
       p = 1
    pos_score = pos_score/p
    neg_score = neg_score/p
    score =(pos_score-neg_score)
    return score,pos_score,neg_score

In [8]:
def negscore(p):
    s = swn.senti_synsets(p)
    p = len(s)
    pos_score=0
    neg_score=0
    for i in s:
        pos_score = pos_score + i.pos_score()
        neg_score = neg_score + i.neg_score()
    if p == 0:
       p = 1
    pos_score = pos_score/p
    neg_score = neg_score/p
    score =(pos_score-neg_score)
    score = score*(-1)
    return score,pos_score,neg_score

In [9]:
def scoring(p):
    fscore = p[0]
    pscore = p[1]
    nscore = p[2]
    return fscore,pscore,nscore

In [10]:
def aspect_check_dict(w):    #check word in aspect list
    global aspect_list
    if w in aspect_list:
        return True
    return False

In [11]:
def neg_check_dict(w):    #check word in aspect list
    global negative_list
    if w in negative_list:
        return True
    return False

In [12]:
file = open('input.txt', "r")
dl = open('dependencylist.txt', "w")
pt = open('parsetree.txt', "w")
dependency = open("op.csv", "w")
negative = open("negop.csv","w")

In [13]:
aspect_list = ['screen','battery','time','price','app','camera','quality','card','life','service','button','network','software','wifi',
               'music','keyboard','memory','sim','video','power','message','size','case']
negative_list = ['don\'t', 'not', 'never','less','no','nothing','nowhere','hardly','barely','scarcely','nobody','none']
flag = ""
negate = ""

In [14]:
input = file.readline()

In [15]:
while input:
    input = input.lower()
    #print input
    #print ('\n')
    dependencynlp = dependency_parser.raw_parse(input)
    for line in dependencynlp:
        x = list(line.triples())
        dl.write(str(x)+'\n')
        print x
        #print('\n')
        y= [parse.tree() for parse in dependency_parser.raw_parse(input)]
        pt.write(str(y)+'\n')
        #print('\n')
        for arg1, pred, arg2 in x:
            word1, pos1 = arg1
            word2, pos2 = arg2
            if pos1.startswith('JJ') and pos2.startswith('RB') and pred == 'neg':
                    if neg_check_dict(arg2[0]):
                        flag = word1
                        negate = word2
                        print flag
                        s = scoring(negscore(arg1[0]))
                        negative.write(str(arg2[0])+","+str(arg1[0])+","+str(s[0])+","+str(s[1])+","+str(s[2])+","+str(pred))
                        negative.write("\n")
        for arg1, pred, arg2 in x:
            word1, pos1 = arg1
            word2, pos2 = arg2
            if pos1.startswith('JJ') and pos2.startswith('NN') and pred == 'nsubj':
                    if aspect_check_dict(arg2[0]) and arg2[0]!= 'i':
                        if word1 == flag:
                            s = scoring(negscore(arg1[0]))
                            dependency.write(str(arg2[0])+","+str(negate+arg1[0])+","+str(s[0])+","+str(s[1])+","+str(s[2])+","+str(pred))
                            dependency.write("\n")
                        else:
                            s = scoring(score(arg1[0]))
                            dependency.write(str(arg2[0])+","+str(arg1[0])+","+str(s[0])+","+str(s[1])+","+str(s[2])+","+str(pred))
                            dependency.write("\n")      
            if pos1.startswith('VBP') and pos2.startswith('NN') and pred == 'dobj':
                    if aspect_check_dict(arg2[0]) and arg2[0]!= 'i':
                        s = scoring(score(arg1[0]))
                        dependency.write(str(arg2[0])+","+str(arg1[0])+","+str(s[0])+","+str(s[1])+","+str(s[2])+","+str(pred))
                        dependency.write("\n")
            if pos1.startswith('NN') and pos2.startswith('JJ') and pred == 'amod':
                    if aspect_check_dict(arg1[0]) and arg1[0] != "i":
                        if word2 == flag:
                            s = scoring(negscore(arg2[0]))
                            dependency.write(str(arg1[0])+","+str(negate+arg2[0])+","+str(s[0])+","+str(s[1])+","+str(s[2])+","+str(pred))
                            dependency.write("\n")
                        else:
                            s = scoring(score(arg2[0]))
                            dependency.write(str(arg1[0])+","+str(arg2[0])+","+str(s[0])+","+str(s[1])+","+str(s[2])+","+str(pred))
                            dependency.write("\n")
            if pos1.startswith('JJ') and pos2.startswith('RB') and pred == 'advmod':
                    if aspect_check_dict(arg2[0]) and arg2[0]!= 'i':
                        if word1 == flag:
                            s = scoring(negscore(arg1[0]))
                            dependency.write(str(arg2[0])+","+str(negate+arg1[0])+","+str(s[0])+","+str(s[1])+","+str(s[2])+","+str(pred))
                            dependency.write("\n")
                        else:
                            s = scoring(score(arg1[0]))
                            dependency.write(str(arg2[0])+","+str(arg1[0])+","+str(s[0])+","+str(s[1])+","+str(s[2])+","+str(pred))
                            dependency.write("\n")
    flag = ""
    negate = ""
    input = file.readline()
file.close()
dependency.close()
dl.close()
pt.close()
negative.close()

[((u'has', u'VBZ'), u'dep', (u'good', u'JJ')), ((u'good', u'JJ'), u'nsubj', (u'screen', u'NN')), ((u'screen', u'NN'), u'det', (u'the', u'DT')), ((u'good', u'JJ'), u'cop', (u'is', u'VBZ')), ((u'good', u'JJ'), u'ccomp', (u'nice', u'JJ')), ((u'nice', u'JJ'), u'cc', (u'but', u'CC')), ((u'nice', u'JJ'), u'nsubj', (u'keyboard', u'NN')), ((u'keyboard', u'NN'), u'det', (u'the', u'DT')), ((u'nice', u'JJ'), u'cop', (u'is', u'VBZ')), ((u'nice', u'JJ'), u'neg', (u'not', u'RB')), ((u'has', u'VBZ'), u'nsubj', (u'it', u'PRP')), ((u'has', u'VBZ'), u'dobj', (u'battery-life', u'NN')), ((u'battery-life', u'NN'), u'amod', (u'slow', u'JJ')), ((u'has', u'VBZ'), u'cc', (u'and', u'CC')), ((u'has', u'VBZ'), u'conj', (u'has', u'VBZ')), ((u'has', u'VBZ'), u'dobj', (u'memory', u'NN')), ((u'memory', u'NN'), u'amod', (u'low', u'JJ'))]
nice
